# Benchmark Analysis with Cycle 2 Output: Surface Metrics for FESOM using [`dask`](https://www.dask.org)
> Paul Gierz  
> AWI Scientific Computing  
> pgierz@awi.de  


This notebook covers how to use [`dask`](https://www.dask.org) in conjunction with the SLURM Batch System on Levante to quickly compute and plot SST and SSS means. We will use the *heavy* simulation `tco3999-ng5`.


In [ ]:
# Some benchmarking for timing:
import datetime
start_time = datetime.datetime.now()

In [ ]:
# Import several libraries, all available in the NextGEMS Hackathon Cycle2 Kernel
import pathlib
import gribscan
import intake
import xarray as xr
import dask
import pandas as pd
import geoviews as gv
import holoviews as hv
import cmocean
import cartopy.crs as crs
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import hvplot.xarray
import math
from holoviews.operation.datashader import datashade, shade, dynspread, spread, rasterize
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
import panel as pn
pn.extension()

In [ ]:
# Open the dataset using the intake catalog:
catalog_file = "../catalog.yaml"
cat = intake.open_catalog(catalog_file)
run = cat.FESOM["tco3999-ng5"]

In [ ]:
# Convert both the data and the grid to Dask arrays
data = run.original_2d.to_dask()
grid = run.node_grid.to_dask()
data

In [ ]:
cluster = SLURMCluster(
    name='dask-cluster', 
    cores=128,
    memory=f"{8000 * 64 * 0.90} MB",  # 230GB
    project="bm1235",
    queue="compute",
    interface='ib0',
    walltime='08:00:00',
    job_extra=["-o dask-worker-%j.log", "-e dask-worker-%j.err", "--verbose"],
)

In [ ]:
print(cluster.job_script())

In [ ]:
# Scale to 15 compute nodes
cluster.scale(jobs=15)

In [ ]:
client = Client(cluster)

In [ ]:
# Show the client object to get a sense for what is there.
# As workers are added or removed, the dashboard might hang!
client

In [ ]:
sst_mean = data.sst.mean(dim="time")
sst_mean

In [ ]:
%%time
sst_mean_computed = sst_mean.compute()

In [ ]:
sst_mean_computed

In [ ]:
cluster.scale(jobs=0)

In [ ]:
# Rename the dimension
sst_mean_computed = sst_mean_computed.rename({"grid_size": "nod2"})

In [ ]:
# Merge the grid and sst means
sst_mean_computed = xr.merge([grid, sst_mean_computed])

In [ ]:
sst_mean_computed

In [ ]:
sst_mean_computed.sst

In [ ]:
def fesom_2d_sfc_field_hover(ds, varname="sst"):
    da = getattr(ds, varname)
    df = da.to_dataframe()
    df.index.name = "Node ID"
    df["Longitude (˚E)"] = ds.grid_center_lon
    df["Latitude (˚N)"] = ds.grid_center_lat 
    points = gv.Points(df)
    def filter_points(points, x_range, y_range):
        if x_range is None or y_range is None:
            return points
        return points[x_range, y_range]

    def hover_points(points, threshold=20000):
        if len(points) > threshold:
            # Do not return any points at all
            return points.iloc[:0]
        return points

    range_stream = hv.streams.RangeXY(source=points)
    streams=[range_stream]

    filtered = points.apply(filter_points, streams=streams)
    shaded = datashade(filtered, width=400, height=400, streams=streams)
    hover = filtered.apply(hover_points)

    dynamic_hover = (hover).opts(gv.opts.Points(tools=['hover'], alpha=0.1, hover_alpha=1, hover_line_color="red", size=10, projection=crs.PlateCarree(), color="black",))
    return dynamic_hover
    

In [ ]:
ds = sst_mean_computed
varname = "sst"
da = getattr(ds, varname)
df = da.to_dataframe()
df.index.name = "Node ID"
df["Longitude (˚E)"] = ds.grid_center_lon
df["Latitude (˚N)"] = ds.grid_center_lat 


In [ ]:
node0 = ds.triag_nodes.isel(Three=0).astype(int)
node1 = ds.triag_nodes.isel(Three=1).astype(int)
node2 = ds.triag_nodes.isel(Three=2).astype(int)

In [ ]:
da_node0 = da.isel(grid_size=node0-1).rename({"ntriags": "node0"})
da_node1 = da.isel(grid_size=node1-1).rename({"ntriags": "node1"})
da_node2 = da.isel(grid_size=node2-1).rename({"ntriags": "node2"})

In [ ]:
array = xr.DataArray([da_node0, da_node1, da_node2], dims=("nod2", "ntriags"))

In [ ]:
sst_on_elements = array.mean(dim="nod2")

In [ ]:
sst_on_elements

In [ ]:
df = pd.DataFrame({"Node 1": node0 - 1, "Node 2": node1 - 1 ,  "Node 3": node2 - 1, "SST": sst_on_elements})
df.index.name = "Element ID"
df.head()

In [ ]:
nodes = gv.Points((ds.grid_center_lon, ds.grid_center_lat))

In [ ]:
nodes.shape

In [ ]:
trimesh = gv.TriMesh((df, nodes)).redim(
        x="Longitude (˚E)", y="Latitude (˚N)", z="Depth (m**2)"
)

In [ ]:
trimesh.shape

In [ ]:
projection = crs.PlateCarree()
projected_trimesh = gv.project(trimesh, projection=projection)

In [ ]:
projected_trimesh.shape

In [ ]:
FESOM_raster = rasterize(projected_trimesh).opts(
        cmap=cmocean.cm.thermal,
        height=600,
        width=900,
        projection=projection,
        colorbar=True,
        colorbar_position="bottom",
        clabel="Sea Surface Temperature (˚C)",
        bgcolor="darkgray",
        color_levels=250,
        clim=(-2, 30),
        cformatter="%.0f",
        title="FESOM SST with {:,} 2d Nodes".format(len(ds.grid_center_lon)),
    )

In [ ]:
dyn_hover = fesom_2d_sfc_field_hover(sst_mean_computed)

In [ ]:
dyn_hover * FESOM_raster

In [ ]:
plot_finished = datetime.datetime.now()

In [ ]:
print(f"Done in {plot_finished - start_time}")